In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
#!wget https://github.com/owid/co2-data/raw/master/owid-co2-data.csv -O ../data/owid-co2-data.csv
raw_data = pd.read_csv("../data/owid-co2-data.csv")

#!wget https://github.com/owid/co2-data/raw/master/owid-co2-codebook.csv -O ../data/owid-co2-codebook.csv
raw_data_info = pd.read_csv("../data/owid-co2-codebook.csv").set_index("column").description
raw_data_source = pd.read_csv("../data/owid-co2-codebook.csv").set_index("column").source

In [ ]:
import plotly.express as px
import altair as alt

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# Define column groups

In [ ]:
def cols(filter_term = None):
    if filter_term is None:
        return raw_data.columns
    else:
        return [c for c in raw_data.columns if filter_term in c]
        
col_keys = ["country", "year"]
cols_per_capita = cols("capita")
cols_relative = cols("relative")
cols_cumulative = cols("cumulative")
cols_per_gdp = cols("per_gdp")
cols_share = cols("share")
cols_not_calculated = raw_data_info.drop(cols_per_capita + cols_relative + cols_cumulative + cols_share + cols_per_gdp)
cols_co2_sources = ["cement_co2", "coal_co2", "flaring_co2", "gas_co2", "oil_co2", "other_industry_co2"]

# Prepare other DataFrames

In [ ]:
countries = raw_data[~raw_data.iso_code.fillna("OWID").str.contains("OWID")]

In [ ]:
#!wget https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv -O ../data/continent.csv
continent_mapping = pd.read_csv("../data/continent.csv").set_index("alpha-3").region.rename("continent")
countries = countries.join(continent_mapping, on="iso_code").dropna(subset=["continent"])

In [ ]:
# Loaded from https://ourworldindata.org/grapher/world-banks-income-groups
income_groups = pd.read_csv("../data/world-banks-income-groups.csv").rename(columns={"Income classifications (World Bank (2017))": "income_group"})
income_groups = income_groups.append(pd.concat([income_groups.query("Year==2016").assign(Year=year) for year in [2017, 2018, 2019, 2020, 2021]]))

countries = countries.join(income_groups.set_index(["Entity", "Year"]), on=["country", "year"])

In [ ]:
not_countries = raw_data[raw_data.iso_code.fillna("OWID").str.contains("OWID")]
world = raw_data.query("country=='World'")
continents = raw_data[raw_data.country.isin(["Europe", "North America", "Asia", "Africa", "South America"])]
incomes = raw_data[raw_data.country.str.contains("-income countries")]

# Prepare Filters

In [ ]:
filter_most_recent = lambda x: x.year == 2020